In [ ]:
!python rag_final.py

/content/rag_final.py:4: LangChainDeprecationWarning: Importing FAISS from langchain.vectorstores is deprecated. Please replace deprecated imports:

>> from langchain.vectorstores import FAISS

with new imports of:

>> from langchain_community.vectorstores import FAISS
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.vectorstores import FAISS
2025-06-19 11:35:13.001585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750332913.020768   28476 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750332913.026673   28476 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when on

In [ ]:
#rag_final.py
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
)

# 1. 모델 설정 (4bit 양자화)
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    llm_int8_enable_fp32_cpu_offload=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
)

text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=2048,
    do_sample=True, #경고 무시하기 위해 true로 설정
    repetition_penalty=1.03,
    pad_token_id=tokenizer.eos_token_id,
)
llm = HuggingFacePipeline(pipeline=text_generator)

# 2. 데이터 불러오기
csv_path = "/content/drive/MyDrive/ColabNotebooks/cleaned_noimage_.csv"
df = pd.read_csv(csv_path, encoding="ISO-8859-1")

# 3. 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
combined_texts = (df["question"].fillna("") + " " + df["answer"].fillna("")).tolist()
docs = text_splitter.create_documents(combined_texts)

# 4. 벡터스토어
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 5. 프롬프트 템플릿
prompt_template = """
제공된 다음 문맥 정보를 바탕으로, 사용자 질문에 대한 **한국어로 된 간결한 답변**만 작성해주세요.
다른 정보나 서론, 추가 설명은 일절 포함하지 마세요. 답변은 오직 한국어로만 구성되어야 합니다.

문맥:
{context}

사용자 질문: {question}

답변:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# 6. QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    input_key="question",
    chain_type_kwargs={"prompt": PROMPT},
)

# 7. 질의 실행
if __name__ == "__main__":
    while True:
        user_query = input("\n❓ 질문 입력 (종료: exit): ")
        if user_query.strip().lower() == "exit":
            print("\n🔚 종료합니다.")
            break

        result = qa_chain.invoke({"question": user_query})
        response_text = result['result']

        if "답변:" in response_text:
            cleaned = response_text.split("답변:", 1)[1].strip()
        else:
            cleaned = response_text.strip()

        print("\n🟢 답변:", cleaned)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



❓ 질문 입력 (종료: exit): URScript에서 MoveJ 명령어에 블렌드 반경을 설정하려면 어떻게 해야 하나요?


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🟢 답변: MoveJ 명령어에 블렌드 반경을 설정하려면, 다음과 같이 작성하시면 됩니다.
```
movej(joint_positions, joint_velocities, joint_accelerations, blend_radius)
```
blend\_radius는 블렌드 반경을 의미하며, 실수 값으로 설정할 수 있습니다.

❓ 질문 입력 (종료: exit): URCap 개발 시 C207A0 오류는 무엇을 의미하나요?


...


🟢 답변: 로봇을 초기화하는 가장 좋은 방법은 로봇의 초기 위치로 이동하고 로봇의 모든 잠재적인 작업을 중지하는 것입니다. 이를 위해서는 MoveJ 또는 MoveL 명령을 사용하여 로봇의 초기 위치로 이동시키고, 로봇의 모든 작업을 중지하기 위해서는 StopJ 또는 StopL 명령을 사용하면 됩니다.

❓ 질문 입력 (종료: exit): URCap 개발 시 C207A0 오류는 무엇을 의미하나요?


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🟢 답변: C207A0 오류는 URCap 개발 중 발생할 수 있는 일반적인 오류입니다. 이 오류는 일반적으로 URCap 개발 중 사용자가 사용하지 않는 코어 패키지를 사용하여 URCap을 구동하려는 시도로 인해 발생합니다. 이 오류를 해결하기 위해서는 사용하지 않는 코어 패키지를 제거하거나 사용하지 않는 코어 패키지를 대체하는 방법을 찾아야 합니다.

❓ 질문 입력 (종료: exit): UR5e의 0번 포트에서 아날로그 입력을 읽으려면 어떻게 하나요?


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🟢 답변: UR5e의 0번 포트는 아날로그 입력을 받지 않습니다. 아날로그 입력을 받기 위해서는 1번 포트(A) 또는 2번 포트(B)를 사용하십시오. 각 포트에는 0~10V 범위의 입력이 가능합니다.

❓ 질문 입력 (종료: exit): exit

🔚 종료합니다.


In [ ]:
!pip install -U langchain-huggingface

In [ ]:
# rag.py

import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
)
from langchain.llms import HuggingFacePipeline

_llm = None
_embeddings = None
_retriever = None
_qa_chain = None

def load_rag_components():
    """
    RAG 시스템에 필요한 모든 컴포넌트(LLM, 임베딩 모델, 벡터스토어, QA 체인)를 로드합니다.
    이 함수는 외부에서 호출되어야 하며, 컴포넌트가 로드되지 않았을 때만 실행됩니다.
    """
    global _llm, _embeddings, _retriever, _qa_chain

    if _qa_chain is not None:
        return True

    # 1.1. LLM 설정 (4bit 양자화)
    model_id = "mistralai/Mistral-7B-Instruct-v0.3"
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
    )

    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        _model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=quantization_config,
            device_map="auto",
        )
        text_generator = pipeline(
            "text-generation",
            model=_model,
            tokenizer=tokenizer,
            max_new_tokens=2048,
            do_sample=True,
            repetition_penalty=1.03,
            pad_token_id=tokenizer.eos_token_id,
        )
        _llm = HuggingFacePipeline(pipeline=text_generator)
    except Exception as e:
        return False

    # 1.2. 데이터 불러오기
    csv_path = "/content/drive/MyDrive/MyDrive/ColabNotebooks/cleaned_noimage_.csv"
    try:
        df = pd.read_csv(csv_path, encoding="ISO-8859-1")

    except FileNotFoundError:

        return False
    except Exception as e:

        return False

    # 1.3. 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
    combined_texts = (
        df["question"].fillna("") + " " + df["answer"].fillna("")
    ).tolist()
    docs = text_splitter.create_documents(combined_texts)
    print(f"총 {len(docs)}개의 문서 청크 생성.")

    # 1.4. 벡터스토어
    try:
        _embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        _vectorstore = FAISS.from_documents(docs, _embeddings)
        _retriever = _vectorstore.as_retriever(search_kwargs={"k": 5})

    except Exception as e:
        return False

    # 1.5. 프롬프트
    prompt_template = """
    제공된 다음 문맥 정보를 바탕으로, 사용자 질문에 대한 **한국어로 된 간결한 답변**만 작성해주세요.
    다른 정보나 서론, 추가 설명은 일절 포함하지 마세요. 답변은 오직 한국어로만 구성되어야 합니다.

    문맥:
    {context}

    사용자 질문: {question}

    답변:
    """
    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"],
    )

    # 1.6. QA 체인
    _qa_chain = RetrievalQA.from_chain_type(
        llm=_llm,
        chain_type="stuff",
        retriever=_retriever,
        return_source_documents=True,
        input_key="question",
        chain_type_kwargs={"prompt": PROMPT},
    )
    return True

def get_qa_chain():
    """
    초기화된 QA 체인 인스턴스를 반환합니다.
    만약 아직 로드되지 않았다면, load_rag_components()를 호출하여 로드를 시도합니다.
    """
    if _qa_chain is None:
        if not load_rag_components():
            return None
    return _qa_chain


if __name__ == "__main__":
    if get_qa_chain():
        user_query = "협동로봇의 주요 안전 수칙은 무엇인가요?"
        result = _qa_chain.invoke({"question": user_query})

        generated_text = result['result']
        answer_prefix_ko = "답변:"
        cleaned_answer = ""
        if answer_prefix_ko in generated_text:
            cleaned_answer = generated_text.split(answer_prefix_ko, 1)[1].strip()
        else:
            cleaned_answer = generated_text

        # print(f"\n--- 테스트 답변 ---") # 제거
        # print(cleaned_answer) # 제거
        # print("\n--- 참고 문서 ---") # 제거
        # for i, doc in enumerate(result['source_documents']): # 제거
        #     print(f"- 문서 {i+1}: {doc.page_content[:150]}...") # 제거

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
)
from langchain.llms import HuggingFacePipeline

# 전역 변수
_llm = None
_embeddings = None
_retriever = None
_qa_chain = None

def load_rag_components():
    global _llm, _embeddings, _retriever, _qa_chain

    if _qa_chain is not None:
        return True  # 이미 로드됨

    # 1. 모델 설정
    model_id = "mistralai/Mistral-7B-Instruct-v0.3"
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
    )

    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=quant_config,
            device_map="auto"
        )
        text_gen = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=2048,
            do_sample=False,
            repetition_penalty=1.03,
            pad_token_id=tokenizer.eos_token_id
        )
        _llm = HuggingFacePipeline(pipeline=text_gen)
    except Exception as e:
        print(f" 모델 로딩 오류: {e}")
        return False

    # 2. CSV 데이터 불러오기
    csv_path = "/content/drive/MyDrive/ColabNotebooks/cleaned_noimage_.csv"
    try:
        df = pd.read_csv(csv_path, encoding="ISO-8859-1")
    except Exception as e:
        print(f" CSV 로딩 오류: {e}")
        return False

    # 3. 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
    combined_texts = (df["question"].fillna("") + " " + df["answer"].fillna("")).tolist()
    docs = text_splitter.create_documents(combined_texts)
    print(f"📄 총 {len(docs)}개의 문서 청크 생성됨.")

    # 4. 벡터스토어 및 임베딩
    try:
        _embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        vectorstore = FAISS.from_documents(docs, _embeddings)
        _retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    except Exception as e:
        print(f" 벡터스토어 구축 오류: {e}")
        return False

    # 5. 프롬프트 템플릿
    prompt_template = """
제공된 다음 문맥 정보를 바탕으로, 사용자 질문에 대한 **한국어로 된 간결한 답변**만 작성해주세요.
다른 정보나 서론, 추가 설명은 일절 포함하지 마세요. 답변은 오직 한국어로만 구성되어야 합니다.

문맥:
{context}

사용자 질문: {question}

답변:
"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    # 6. QA 체인 구성
    _qa_chain = RetrievalQA.from_chain_type(
        llm=_llm,
        chain_type="stuff",
        retriever=_retriever,
        return_source_documents=False,
        input_key="question",
        chain_type_kwargs={"prompt": PROMPT},
    )

    return True

def get_qa_chain():
    global _qa_chain
    if _qa_chain is None:
        if not load_rag_components():
            return None
    return _qa_chain

if __name__ == "__main__":
    print("✅ RAG 시스템 초기화 중...")
    if get_qa_chain():
        print("✅ 시스템 준비 완료! 질문을 입력하세요.")
        while True:
            user_query = input("\n❓ 질문 입력 (exit 입력 시 종료): ")
            if user_query.lower() in ["exit", "quit", "종료"]:
                print("👋 종료합니다.")
                break

            result = _qa_chain.invoke({"question": user_query})
            generated_text = result['result']

            # 답변 정제
            answer_prefix = "답변:"
            if answer_prefix in generated_text:
                cleaned_answer = generated_text.split(answer_prefix, 1)[1].strip()
            else:
                cleaned_answer = generated_text

            print("🟢 답변:", cleaned_answer)
    else:
        print("❌ 시스템 로딩 실패. 확인 후 다시 실행하세요.")


✅ RAG 시스템 초기화 중...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipython-input-5-2308166378.py:54: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  _llm = HuggingFacePipeline(pipeline=text_gen)
/tmp/ipython-input-5-2308166378.py:75: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langch

📄 총 561개의 문서 청크 생성됨.
✅ 시스템 준비 완료! 질문을 입력하세요.


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🟢 답변: 로봇을 초기화하는 가장 좋은 방법은 로봇의 초기 위치로 이동하고 로봇의 모든 잠재적인 작업을 중지하는 것입니다. 이를 위해서는 MoveJ 또는 MoveL 명령을 사용하여 로봇의 초기 위치로 이동시키고, 로봇의 모든 작업을 중지하기 위해서는 StopJ 또는 StopL 명령을 사용하면 됩니다.


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `as` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `as`


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# RAG

RAG